## Example for our package

We are going to generate a multivariate time series and then look for anomalies in it. 

In this example, we shall suppose that the probability that a data point in the time series is an anomaly is called __my_tau__. Here we have set it equal to 0.01, but feel free to change it (and any other variables) to some other value. 

In [ ]:
my_tau = 0.01

Next we can choose how many data dimensions the time series should have.

In [ ]:
my_d = 10

Data is going to be shown to the anomaly models in batches of size __B__. Since we are after some stability in scores from anomaly models that may be contextual (e.g., the Isolation Forest anomaly score for a given point depend on the values of the other points considered with it), we suggest that __B__ should be relatively large (e.g., 500). Note also that there is a parameter called __n_data_min__ in the init_super_sad() and super_sad() functions that you will run which has a default value of 100. If you choose __B__ less than this value, it will stop those functions and output an error (unless you also change the default value of __n_data_min__ when you call these functions). 

In [ ]:
B = 500

Next we can choose how many batches of data, called __n_loops__, that we are going to simulate. In the real world, this may not be a fixed number (e.g., in a factory that never stops running), so if your data is _a priori_ not fixed in number, you would have to modify the script below to take that into account. Since we are simulating data here, it is easiest to simulated a fixed total amount from the beginning. 

In [ ]:
n_loops=75

We shall suppose that there is one further batch of size __B__ of "old data", that is, the first __B__ points of the time series have already been generated and we may or may not come with some information about the nominal or anomaly status of some or all of those points.

In [ ]:
n_old = B

You could actually change the value of __n_old__ to something other than __B__ but we recommend against it. The main reason is that this creates a batch size that is different to all of the other batch sizes, and if the difference is too great, it might do something weird with the nominal and anomaly score distributions of the anomaly detectors, which could upset the supervised classifier. 

In any case, the total number of data points __n__ we will simulate is then given by: 

In [ ]:
n = n_old + B*n_loops

Next, we choose the maximum number of data points __n_send__ to send to an expert for labeling as nominal or anomaly from each new batch. Note that we say "maximum" because inside the function __init_sad()__, the data points sent to the expert come from two sources, obtained in parallel: (i) anomaly predictions from the supervised classifier, and (ii) data points chosen by the active learner. We will see below that the number of points in total these two sources send is meant to sum to __n_send__. However, what can sometimes happen is that they independently select one or more of the _same_ points. When this happens, less than __n_send__ data points are sent to the expert. 

In [ ]:
n_send = 5

Since we are simulating the data here, and, in the end, learning a supervised classifier to predict "nominal" or "anomaly" on future data, we are also able to simulate an independent time series of the same length (excluding the __n_old__ old data) and use it to estimate how well our method is performing (e.g., how good is the supervised classifier we learn in terms of the area under the ROC curve (AUC) or some other criterion).

In [ ]:
n_ext = B*n_loops

Next, we shall set the value of the percentage __u__ of the "old data" for which we suppose we known its label (nominal or anomaly). Thus we start with __u x n_old__ already-labeled data points. 

In [ ]:
u = 0.2

We now move to the parameters required to simulate the actual time series data points. There are four parameters required for these simulations: the means of three __my_d__-dimensional Gaussian random variables, and __c_anom__, a constant (see below).  

In [ ]:
import numpy as np

mean_G1=np.full(my_d, 5) 
mean_G2=np.full(my_d, 5.75) 
mean_G3=np.full(my_d, 6.5)
c_anom=0.01

Before continuing we can set a random seed for the reproducibility of the results below (optional).

In [ ]:
np.random.seed(123456789)

The Gaussians __G1__ and __G3__ will have identity covariance matrices, while the Gaussian __G2__ will have an identity covariance matrix scaled by __c_anom__.

The data is generated as follows:
- The jump from one time point to the next is generated i.i.d. from a mixture distribution of the three Gaussians. Precisely, the next jump comes from __G1__ with probability (1 - __my_tau__)/2, from  __G2__ with probability __my_tau__, and from __G3__ with probability (1 - __my_tau__)/2. The jumps from __G2__ correspond to anomalies, and occur __my_tau__ % of the time.
- This jump is then multiplied by -1 with probability 0.5, and the result is added to the previous value of the time series, except for the first data point, where it is added to __my_d__-dimensional 0, i.e., the first jump equals the first data point.

Hence, with the default parameters for __G1__, __G2__, and __G3__ chosen above, this time series is a kind of random walk where the "smallest" jumps mostly lead to nominal points, the "medium" jumps (may) lead to anomaly point, and the "large" jumps mostly lead to nominal points too. 

Note however that with the default values for the three means, these three distributions are highly overlapping, and so "medium" jump does by no means definitively imply an anomaly. There is a plot below which illustrates this. But in order to get to that, we first have to actually generate the data. The function we have written to do so (included in this library) is named __generate_time_series()__ and is called as follows: 

In [ ]:
def generate_time_series(my_tau, d, n, 
                         mean_G1=None, mean_G2=None, mean_G3=None, 
                         cov_G1=None, cov_G2=None, cov_G3=None, 
                         c=1,c_anom = 1):
    """
    Generates a time series with JUMPS (not the data itself) drawn from three d-dimensional 
    Gaussians G1, G2, and G3. G1 and G3 are nominal data while G2 is anomalies. 
    (So this is a kind of random walk time series.) Jumps are further multiplied by -1 
    with proability 0.5.
    
    Parameters:
    - tau: Probability of selecting G2.
    - d: Data dimensionality.
    - n: Number of data points to generate.
    - mean_G1, mean_G2, mean_G3: Means of the Gaussians (default: [1,...,1], [2,...,2], [3,...,3]).
    - cov_G1, cov_G2, cov_G3: Covariance matrices (default: c * Identity matrix).
    - c: Scaling factor for identity covariance matrices (default: 1).
    - c_anom: a constant to multiply the anomaly identity covariance matrix by.

    Returns:
    - time_series: Generated time series (n x d array).
    - Y: Labels indicating whether each point came from G2 (1) or from G1/G3 (0).
    """
    # Default mean values
    if mean_G1 is None:
        mean_G1 = np.ones(d)
    if mean_G2 is None:
        mean_G2 = np.full(d, 2)
    if mean_G3 is None:
        mean_G3 = np.full(d, 3)
    
    # Default covariance matrices
    if cov_G1 is None:
        cov_G1 = c * np.eye(d)
    if cov_G2 is None:
        cov_G2 = c_anom * np.eye(d)
    if cov_G3 is None:
        cov_G3 = c * np.eye(d)
    
    # Initialize time series storage
    time_series = np.zeros((n, d))
    
    # Initialize Y labels
    Y = np.empty(n, dtype=int)

    # First value is generated independently
    probabilities = [(1 - my_tau) / 2, my_tau, (1 - my_tau) / 2]
    choice = np.random.choice([1, 2, 3], p=probabilities)

    if choice == 1:
        temp_value = np.random.multivariate_normal(mean_G1, cov_G1)
        Y[0] = 0
    elif choice == 2:
        temp_value = np.random.multivariate_normal(mean_G2, cov_G2)
        Y[0] = 1
    else:
        temp_value = np.random.multivariate_normal(mean_G3, cov_G3)
        Y[0] = 0

    if np.random.rand() < 0.5:
        temp_value *= -1  # Flip sign with probability 0.5

    time_series[0] = temp_value  # First value

    # Generate the rest of the series
    for i in range(1, n):
        choice = np.random.choice([1, 2, 3], p=probabilities)

        if choice == 1:
            temp_value = np.random.multivariate_normal(mean_G1, cov_G1)
            Y[i] = 0
        elif choice == 2:
            temp_value = np.random.multivariate_normal(mean_G2, cov_G2)
            Y[i] = 1
        else:
            temp_value = np.random.multivariate_normal(mean_G3, cov_G3)
            Y[i] = 0

        if np.random.rand() < 0.5:
            temp_value *= -1  # Flip sign with probability 0.5


        time_series[i] = time_series[i - 1] + temp_value  # Cumulative sum

    return time_series, Y

In [ ]:
X, Y = generate_time_series(my_tau=my_tau, d=my_d, n = n, mean_G1=mean_G1, mean_G2=mean_G2, mean_G3=mean_G3, c_anom = c_anom)

We can plot the first __n_old__ points of the first dimension of this data series to see what it looks like. Given the way the three means are defined, all of the dimensions will have relatively similar behavior, so we don't really need to plot them all.  

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.lines as mlines

plt.figure(figsize=(10, 6))

plt.scatter(range(B), X[0:n_old, 0], 
            c=['blue' if y == 0 else 'red' for y in Y[0:n_old]], s=4, marker='o')

# Set axis labels and title
plt.xlabel("Time")
plt.ylabel("Dimension 1 data values")
plt.title("Data dimension 1 with status labels")

# Create custom legend markers
blue_marker = mlines.Line2D([], [], color='blue', marker='o', markersize=5, label="Nominal (Y=0)", linestyle='None')
red_marker = mlines.Line2D([], [], color='red', marker='o', markersize=5, label="Anomaly (Y=1)", linestyle='None')

# Add legend
plt.legend(handles=[blue_marker, red_marker], loc="best")

# Show plot
plt.show()

Notice how the true anomalies in red do not seem particularly "different" to the nominals in blue. This is because in these simulations, anomaly status is with respect to the size of the jump from the previous point, not to the actual data values plotted. Anomalies tend to correspond to "medium" jumps, though this is not easy to see here, since the three jump distributions in the first dimension overlap significantly with the default parameters above (they correspond to the Gaussians $\mathcal{N}(\text{mean\_G1},I_{\text{my\_d}}) \,$, $\mathcal{N}(\text{mean\_G2},\text{c\_anom}\cdot I_{\text{my\_d}})\,$, and $\mathcal{N}(\text{mean\_G3},I_{\text{my\_d}})$.  

We can instead plot the absolute value of the jumps between successive data points:

In [ ]:
# Compute Z values (the absolute values of the jumps)
Z = np.abs(np.diff(X[:, 0], prepend=X[0, 0]))  # Prepend first value to maintain length

# Create standalone figure
plt.figure(figsize=(10, 6))

# Scatter plot of transformed values
plt.scatter(range(n_old), Z[0:n_old], 
            c=['blue' if y == 0 else 'red' for y in Y[0:n_old]], s=4, marker='o')

# Set axis labels and title
plt.xlabel("Time")
plt.ylabel("Absolute jump")
plt.title("Absolute value of the jump between successive data values")

# Create custom legend markers
blue_marker = mlines.Line2D([], [], color='blue', marker='o', markersize=5, label="Nominal (Y=0)", linestyle='None')
red_marker = mlines.Line2D([], [], color='red', marker='o', markersize=5, label="Anomaly (Y=1)", linestyle='None')

# Add legend
plt.legend(handles=[blue_marker, red_marker], loc="best")

# Show plot
plt.show()

Now we can see that anomalies indeed correspond to "medium" absolute jumps. However, we also see that they are in the middle of a sea of nominals that also have "medium" absolute jumps. Despite this, we shall show that with at least one anomaly detector model that outputs scores related to these Euclidean jump distances, we will be able to detect a good number of anomalies nevertheless.

Next, we generate an external validation time series using the same data generation mechanism:

In [ ]:
X_ext, Y_ext = generate_time_series(my_tau=my_tau, d=my_d, n = n_ext,mean_G1=mean_G1, mean_G2=mean_G2, mean_G3=mean_G3,c_anom = c_anom)

Following this, we now have to artificially "hide" the labels of (1 - __u__) % of the "old" data so that we only have access to  __u__ % of the true labels of the "old" data. However, we need to do this in such a way that among the data points that get visible labels, there are exactly some small number __n_lab_anom__ of labeled anomalies. 

Why do we do this? Well, in the real world, if you end up in a situation where there is no old data that is labeled at all, or only some nominals are known, and you have no prior information about what anomalies might "look like" compared to nominals, you are in a very hard place. Without any useful prior knowledge, in for example a factory situation where every time series dimension is a numerical sensor that outputs numbers at each time step, you may be left having to wait for anomaly events (something breaking down, for instance) in the factory, and then a posteriori labeling as "anomaly" the associated time point's multi-sensor data vector, in order to build up a collection of (sensor data, anomaly) pairs to learn from in some way. 

Here, since we are simulating the data, we could easily start off supposing that we know no labels in the "old" data, but without prior knowledge, our active learning phase would have to be a random sample in each batch until we finally found some true anomalies to label as such (since also we cannot do typical binary classification when we have no items to learn on from one of the two classes). Rather than actually coding this as a loop over early batches, we simply pretend, as it were, that through some process, we have _already_ obtained a small number of labeled anomalies: a subset of the "old" data. 

Here, we suppose there are two labeled anomalies, but you can change this value if you like (and see below for possible issues) though of course it has to be a positive integer.

In [ ]:
n_lab_anom = 2

We do now need to check that there are at least __n_lab_anom__ true anomalies in the "old" data, otherwise there is no way we'll be able to randomly select __u__ % of the old data and get __n_lab_anom__ true anomalies to give visible labels to!

In [ ]:
if np.sum(Y[:n_old] == 1) < n_lab_anom:
    raise ValueError(f"There have to be at least {n_lab_anom} true anomalies in the old data but there are only {np.sum(Y[:B] == 1)} of them.")

Note that, more generally, the value of __n_lab_anom__ also has to make sense with respect to the value of __u__ and __n_old__. By this, we mean that what we do here is randomly choose __u__ % of the first __n_old__ data points in __X__ and count how many of them are true anomalies (i.e., have label 1 in __Y__). If this number is exactly __n_lab_anom__, we have found our labeled subset of the "old data" and can move on. However, if not, we randomly choose __u__ % of the "old" data again, and continue to do so until we get exactly __n_lab_anom__ anomalies with visible labels as such. It is quite easy to choose badly the fixed values for __n_lab_anom__ and __u__ so that, with respect also to the value of __n_old__, it is almost impossible to obtain exactly __n_lab_anom__ data points that are anomalies via this random selection process. The default parameters here work fine, but if you change them, we have added a stopping condition __num_tries_max__ below so that this process doesn't go on forever. If the stopping condition is hit, you should probably have another go at your choice of parameters __u__ and __n_lab__anom__.

In [ ]:
#Maximum number of tries to get the labeled subset we want:
num_tries_max = 100

#Number of tries so far:
num_tries_so_far = 0

#Initialize the variable n_remaining_anomalies with a value different to n_lab_anom
n_remaining_anomalies = 0

#Calculate how many labels to show and how many to hide:
n_hide = int(np.ceil((1-u)*n_old)) # if u > 0 then at least one label will be shown due to ceiling function
n_hide_pc = (n_hide/n_old)*100

#Two values used in the print output below:
n_old_anomalies = np.sum(Y[:n_old] == 1)
n_old_nominals = n_old - n_old_anomalies

while (n_remaining_anomalies != n_lab_anom) & (num_tries_so_far < num_tries_max):

    #Update num_tries_so_far:
    num_tries_so_far = num_tries_so_far + 1
    
    #Randomly select n_hide of the n_old data points to mask:
    permute_indices = np.random.permutation(n_old)
    
    # The n_anomalies indices in permute_indices will correspond to the anomalies:
    hide_indices = permute_indices[0:n_hide]
    
    #Fill in Y_muted:
    Y_muted = Y[:n_old].astype(float)
    Y_muted[hide_indices] = np.nan
    n_remaining_anomalies = np.sum(Y_muted == 1)
    n_remaining_nominals = np.sum(Y_muted == 0)
    print('There were',n_old_anomalies,'anomalies and',n_old_nominals,'nominals in the initial data. After randomly masking',n_hide_pc,'% of the initial data, there remain',n_remaining_anomalies,'labeled anomalies and',n_remaining_nominals,'labeled nominals.')

if (n_remaining_anomalies != n_lab_anom) & (num_tries_so_far == num_tries_max):
    raise ValueError(f"You have not obtained a random sample with {n_lab_anom} anomalies in it. Please reconsider your choice of u and n_lab_anom with respect to the value of B. Do not continue until this error message no longer occurs.")
    

We next come to the point in time where we must select an ensemble of unsupervised anomaly detection models to be included in our method. We remark that there are already several such models contained in scikit-learn, see the outlier section of the online Chapter here: https://scikit-learn.org/stable/modules/outlier_detection.html.

We have also written a few extra models that can be used here. You too can create your own unsupervised anomaly detector classes. To do so, you simply have to write the class in such a way that it contains a __score_samples()__ function that can be called. For example, here is a very simple class we wrote called __RandomScore__. It simply takes in a batch of data __X__ and, when called, outputs a random uniform score on [0,1] for each data point (row of __X__). 


    class RandomScore:

        """
        Random score class to return random uniform vectors using the score_samples convention.
        """
        
        def __init__(self):
            pass
        
        def fit(self, X):
            # No actual fitting logic for RandomScore; just a placeholder
            self.X = X
        
        def score_samples(self, X):
            # Assign a random score between 0 and 1 for each data point
            return np.random.rand(X.shape[0])

Note that __fit()__ does not necessarily have to do anything, it is simply there to be consistent with other classes. A slightly more sophisticated model we wrote is called __EuclideanDifferenceAnomalyDetector()__. It doesn't really require __fit()__ either, but unlike the random score model above, it does create scores from the data using score_samples() by calculating the Euclidean distance between consecutive points. 

    class EuclideanDifferenceAnomalyDetector:
    
        def __init__(self):
            """
            Initializes the Euclidean Difference Anomaly Detector.
            This model calculates the Euclidean distance between consecutive rows of the data,
            i.e., the "size" of the jumps between consecutive points. 
            """
            pass
        
        def fit(self, X):
            
            """
            Fits the model to the data (although no actual fitting is necessary here).
            This is just for consistency with the framework.
            
            Parameters:
            - X: The data to fit the model to.
            """
            
            # In this case, there's no fitting process required, so we'll just store X if needed
            self.X = X  # Optional: We store X for future use if needed.
        
        def score_samples(self, X):
            
            """
            Computes the Euclidean distance between consecutive rows and returns the scores.
            
            Parameters:
            - X: The data to compute anomaly scores for (shape: [n_samples, n_features]).
            
            Returns:
            - scores: A NumPy array of scores, where each value is the Euclidean distance 
                      from the previous row.
            """
            
            # Initialize a list to hold the scores
            scores = np.zeros(X.shape[0])
            
            # Calculate the Euclidean distances between consecutive rows
            for i in range(1, X.shape[0]):
                # Euclidean distance between row i and row (i-1):
                dist = np.linalg.norm(X[i] - X[i-1])
                scores[i] = dist
            
            # The first row score is set to the same as the second row score by convention:
            scores[0] = scores[1]
            
            
            return scores

You too can call either of these models since they are part of this library.

Here we are going to build a dictionary of models to use today. Mostly this is easy since usually when we define one of these models, we simply explicitely give any parameters we want it to have inside the parentheses after the function name. However, we have also in the project worked with random LODA projections: (Tomáš Pevny. (2016). Loda: Lightweight on-line detector of anomalies. Machine Learning, 102:275–304, 2016.). For these, we cannot write out their parameters in advance, they must be _randomly_ generated, and furthermore, we can have more than one LODA model if we like. 

Since in general we do not always recommend using LODA projections at all, yet we want to allow users to include them if they want, we have decided to try and allow this possibility when constructing dictionaries of anomaly detection models. Here is the function that builds model dictionaries:

    def Create_Anomaly_Models(d, n_LODA_models=0, additional_models=None):
        """
        Creates a fixed number of LODA models and user-specified anomaly detection models.
        
        Parameters:
        - d: Data dimensionality.
        - n_LODA_models: The number of LODA models to generate (default = 0).
        - additional_models: Dictionary of {model_name: uninitialized model instance}.
    
        Returns:
        - models: Dictionary containing all models.
        """
        
        # Initialize models dictionary
        models = {}
        
        # Loop to create n_LODA_models
        for i in range(n_LODA_models):
            # Increment LODA_index to generate unique model names (Loda_1, Loda_2, ..., Loda_M)
            current_LODA_index = i + 1
            
            # Generate a single LODA model and add it to the models dictionary
            new_model, proj_vectors = LODA_OAT(M=1, d=d, models=None, LODA_index=current_LODA_index)
            
            # Add the newly generated models to the models dictionary
            models.update(new_model)
        
        # Add user-specified models (e.g., IsolationForest, OneClassSVM)
        if additional_models:
            for model_name, model_instance in additional_models.items():
                models[model_name] = model_instance
    
        return models

Note that we have set the default number of LODA models to include to be zero. The user therefore has to modify this parameter explicitly if they want to include LODA models. 

As for all of the other models, they are included via the __additional_models__ variable. Here for instance, let us include a few models in this way: 

In [ ]:
class RandomScore:

    """
    Random score class to return random uniform vectors using the score_samples convention.
    """
    
    def __init__(self):
        pass
    
    def fit(self, X):
        # No actual fitting logic for RandomScore; just a placeholder
        self.X = X
    
    def score_samples(self, X):
        # Assign a random score between 0 and 1 for each data point
        return np.random.rand(X.shape[0])

class EuclideanDifferenceAnomalyDetector:
    
    def __init__(self):
        """
        Initializes the Euclidean Difference Anomaly Detector.
        This model calculates the Euclidean distance between consecutive rows of the data,
        i.e., the "size" of the jumps between consecutive points. 
        """
        pass
    
    def fit(self, X):
        
        """
        Fits the model to the data (although no actual fitting is necessary here).
        This is just for consistency with the framework.
        
        Parameters:
        - X: The data to fit the model to.
        """
        
        # In this case, there's no fitting process required, so we'll just store X if needed
        self.X = X  # Optional: We store X for future use if needed.
    
    def score_samples(self, X):
        
        """
        Computes the Euclidean distance between consecutive rows and returns the scores.
        
        Parameters:
        - X: The data to compute anomaly scores for (shape: [n_samples, n_features]).
        
        Returns:
        - scores: A NumPy array of scores, where each value is the Euclidean distance 
                  from the previous row.
        """
        
        # Initialize a list to hold the scores
        scores = np.zeros(X.shape[0])
        
        # Calculate the Euclidean distances between consecutive rows
        for i in range(1, X.shape[0]):
            # Euclidean distance between row i and row (i-1):
            dist = np.linalg.norm(X[i] - X[i-1])
            scores[i] = dist
        
        # The first row score is set to the same as the second row score by convention:
        scores[0] = scores[1]
        
        
        return scores

In [ ]:
import sklearn.ensemble as skens
import sklearn.neighbors as skneighb
import sklearn.svm as sksvm

additional_models = {
    "IsolationForest": skens.IsolationForest(),
    "OneClassSVM": sksvm.OneClassSVM(),
    "EuclideanDifferenceAnomalyDetector": EuclideanDifferenceAnomalyDetector(),
    "LocalOutlierFactor": skneighb.LocalOutlierFactor(novelty=True),
    "RandomScore": RandomScore(),
}

As you can see, three of these function classes are from scikit-learn, while the other two are classes we wrote earlier.

__Warning__: if you want to use the scikit-learn class LocalOutlierFactor as an anomaly detection, you have to include the argument __novely=True__ ! 

Note that our classes __EuclideanDifferenceAnomalyDetector()__ and __RandomScore()__ do not accept arguments, but all of the scikit-learn classes do. Please refer to https://scikit-learn.org/stable/modules/outlier_detection.html for further details.

Note also that you could, if you like, have more than one model from the same type of class in __additional_models__ ! You just need to choose different parameters to put between the parenthesis. However, you should be careful which supervised classifier you use later on, since the risk is that you could obtain overly correlated scores between versions of the same model, which could for instance affect the convergence of supervised methods such as logistic regression. (A random forest classifier, on the other hand, should not be too affected.)

So, now we can build a dictionary of models using the function __Create_Anomaly_Models()__. We have chosen to include five LODA projection models. We also have to provide the data dimension __d__ as input, even though in reality it is only needed if we are including LODA projection models (since they need to know the data dimension). 

In [ ]:
import acanag.loda_utils as loda

def Create_Anomaly_Models(d, n_LODA_models=10, additional_models=None):

    # dans notre fonction main ::
    
    """
    Creates a fixed number of LODA models and user-specified anomaly detection models.
    
    Parameters:
    - d: Data dimensionality.
    - n_LODA_models: The number of LODA models to generate.
    - additional_models: Dictionary of {model_name: uninitialized model instance}.
        Current options include:
        "IsolationForest": IsolationForest()
        "OneClassSVM": OneClassSVM()
        "EuclideanDifferenceAnomalyDetector": EuclideanDifferenceAnomalyDetector()
        "LocalOutlierFactor": LocalOutlierFactor(novelty=True)
        "RandomScore": RandomScore()
        
        Usage example: additional models = {
                            "IsolationForest": IsolationForest(),
                            "OneClassSVM": OneClassSVM()
                            }

    Returns:
    - models: Dictionary containing all models.
    """
    
    # Initialize models dictionary
    models = {}
    
    # Loop to create n_LODA_models
    for i in range(n_LODA_models):
        # Increment LODA_index to generate unique model names (Loda_1, Loda_2, ..., Loda_M)
        current_LODA_index = i + 1
        
        # Generate a single LODA model and add it to the models dictionary
        new_model, proj_vectors = loda.LODA_OAT(M=1, d=d, models=None, LODA_index=current_LODA_index)
        
        # Add the newly generated models to the models dictionary
        models.update(new_model)
    
    # Add user-specified models (e.g., IsolationForest, OneClassSVM)
    if additional_models:
        for model_name, model_instance in additional_models.items():
            models[model_name] = model_instance

    return models

In [ ]:
models = Create_Anomaly_Models(my_d, n_LODA_models=5, additional_models=additional_models)

Please feel free to look at how models is stored as a dictionary:

In [ ]:
models

At this point, we can also pre-calculate all of the raw anomaly scores on the external validation data set by batches of size __B__ with respect to each of the models in the dictionary, using the __Compute_All_Model_Scores()__ function:

In [ ]:
import acanag.acanag as aaa

new_unweighted_validation_scores = np.empty((np.shape(X_ext)[0],len(models)))

for j in range(n_loops):
    #left and right indices:
    inlef = B*j 
    inrig = B*(j+1)
    new_unweighted_validation_scores[inlef:inrig,:] = aaa.Compute_All_Model_Scores(X_ext[inlef:inrig,:],models)

__Warning__: All of these models do not necessary output scores that are, for example, normalized to be positive and say between 0 and 1. Scores from some models may be negative fully or in part, and one model might give scores close to 0 while another might give scores close to 100. This is important when it comes to choosing the supervised classifier in our framework, since having vastly different scales for anomaly scores might affect convergence or classification quality. (Something like a random forest classifier should not be too affected.)

We are now ready to initialize based on what we know about the old data, using our initialization function __init_super_sad()__:

In [ ]:
X_old = X[:n_old,:]
Y_old = Y_muted.tolist()  

In [ ]:
X_old, X_lab, Y_lab, all_labeled_scores = aaa.init_super_sad(X_old = X_old, Y_old = Y_old, data_dim = my_d, n_data_min = 100, models=models)

We then predefine a list __AAA_AUC__ to stock the evolution of the area under the ROC curve (AUC) over time as successive batches of data are included. By this, we mean that after each batch is added and the expert receives a small number of new items to label, we take the current set of labeled data and their associated scores, and re-train our chosen supervised classifier (see below), before using it to predict the labels of the external data set 

In [ ]:
AAA_AUC = [0]*n_loops

We can now loop through the blocks, running our main function __super_sad()__ in each loop, updating everything of interest as we go. This should take a few minutes to run on a fast laptop.

Several comments about the script below:

- In the function call we have the argument __query_strategy = margin_sampling__. Margin sampling is one of six active learning strategies built in to the Python __modAL__ package, see here: https://modal-python.readthedocs.io/en/latest/content/apireference/uncertainty.html . You can change __margin_sampling__ to one of the other options if you like. 

- In the function call, we have the argument: __supervised_model={'class': RandomForestClassifier, 'params': {'class_weight': {0: 0.999, 1: 0.001}}}__. This means we have chosen our supervised classifier to be a random forest that expects the frequency of anomalies to be 0.001. (Even though we actually simulated the data with a frequency of 0.01 here. But we are pretending that we don't know this frequency, so we don't want to artificially inflate the results quality by putting the true frequency here in the function call.)

- You can of course choose any supervised classifier from scikit-learn that you like. However, note that we do not use the 0-1 predictions here, we use the output from the __predict_proba()__ function (see below), so in order for this to function correctly, the supervised learner you choose from scikit-learn must have __predict_proba()__ also available to it. This is generally the case, but not always. If there is a mismatch, the code below will output an error.

- We believe using random forest is a good idea if the anomaly scores from different anomaly models are wildly different in scale and center, as is the case in this example, since it is not really affected by the scale of different variables.

In [ ]:
import modAL.uncertainty as mdunc
from sklearn.metrics import roc_auc_score

nY1_over_time = []  # List to store number of anomalies detected over time

curr_L_index = n_old # Make sure we correctly define the left index of the start of the new data

for r in range(n_loops): 

    print('We are at batch',r+1,'of',n_loops)
    
    #Get the right-hand side index too:
    curr_R_index = curr_L_index + B
    
    #Get next batch of data:
    X_new = X[curr_L_index:curr_R_index,:] 
    Y_new = Y[curr_L_index:curr_R_index].tolist()

    # Learn from labeled data, propose new predicted anomalies, and propose other data to label:
    X_old, X_lab, all_labeled_scores, indices_to_expert, learned_model, supervised_indices, active_indices =  aaa.super_sad(X_new = X_new, X_old = X_old, X_lab = X_lab, Y_lab = Y_lab, all_labeled_scores = all_labeled_scores, models=models, supervised_model={'class': skens.RandomForestClassifier, 'params': {'class_weight': {0: 0.999, 1: 0.001}}}, query_strategy = mdunc.margin_sampling, n_data_min = 100, n_data_max = 2000, min_n_labeled = 5, n_send=n_send, pc_top = 0.4, min_n_nom=5, min_n_anom=1)
    
    # Pretend to be the expert and add the true labels to the proposed data:
    expert_provided_labels = [Y_new[j] for j in indices_to_expert]
    Y_lab = Y_lab + expert_provided_labels
    
    nY0 = Y_lab.count(0)
    nY1 = Y_lab.count(1)
    print('There are ',nY0,' labeled non-anomalies and ',nY1,' labeled anomalies so far.')

    # Store the count of anomalies detected so far
    nY1_over_time.append(nY1)
    
    #Test the current learned model on the external data in order to calculate the AUC:
    #new_preds = learned_model.predict_proba(new_unweighted_validation_scores)[:, 1]
    if learned_model != None:
        new_preds = learned_model.predict_proba(new_unweighted_validation_scores)[:,1]
        AAA_AUC[r] = roc_auc_score(Y_ext,new_preds)
    else:
        AAA_AUC[r] = 0.5
        
    #Update indices:
    curr_L_index = curr_L_index + B

Notice how this script also tells us how many of the items sent to the expert for labeling have turned out to be true nominals or true anomalies after each loop.

Here is the plot of the AUC over time as more and more batches are examined:

In [ ]:
xx = list(range(1,n_loops+1))

# Plot the column averages
plt.figure(figsize=(10, 6))
plt.plot(xx,AAA_AUC, marker='d')

# Add plot title and labels
plt.title("AUC over time for one trial")
plt.xlabel("Batch", fontsize=14)
plt.ylabel("AUC", fontsize=14)

plt.xticks(ticks=list(range(5, max(xx) + 1, 5)))

# Add grid for better readability
plt.grid(True)

# Show the plot
plt.show()

If you have not changed any of the parameters so far, you should see an AUC plot that seems to increase over time, from around 0.7 to around 0.8, on average (though with some variation from batch to batch).

And now here is a plot of the cumulative number of true anomalies detected over time:

In [ ]:
# Plot the number of labeled anomalies over time
plt.figure(figsize=(10, 6))
plt.plot(xx, nY1_over_time, marker='o', linestyle='-', color='red')

plt.xlabel("Batch number", fontsize=14)
plt.ylabel("Number of anomalies detected", fontsize=14)
plt.title("Cumulative number of labeled anomalies over time", fontsize=16)

plt.xticks(ticks=range(1, n_loops + 1, 5))  # Show every 5 batches on x-axis
plt.grid(True)

plt.show()

One could interpret the part of the curve where it is "parabolic" as the random forest classifier & active learner getting "better and better" over time, though it may only be the classifier, especially as the behavior of the active learner will depend on which type of active learning you tell the function to do. As for where the curve starts to flatten out, in this example since the true anomalies all have scores close to each other, it suggests that the random forest classifier has reached its "best as it can do" level (approximately). It will still make mistakes since the range of scores it is predicting anomalies inside still contains plenty of nominals too, but the fraction of its predictions that are true anomalies becomes on average a constant per batch. This might not be the case if there is some other score range where anomalies exists in number, that we have not detected at all. If the active learner suddenly proposes one of these for labeling to the expert, the supervised classifier will suddenly have to take into account new and quite different information about where anomalies are located, which could affect the above plot in unknown ways. 

If we want to get a better idea of what is happening, "on average", we can simply repeat the same trials above some number of times and take an average of the AUC. If you would like to do this, here is the code (it may take several minutes per trial, so take that into account before launching it). Note that we reuse the same anomaly detector models as above.

All of the code in the main block below is exactly the same code as above.

In [ ]:
n_trials = 2

In [ ]:
all_AAA_AUC = np.zeros((n_trials,n_loops))
all_nY1 = np.zeros((n_trials, n_loops))  # Array to store nY1 values

for curr_trial in range(n_trials):
    print('We are in trial',curr_trial+1,'out of',n_trials) 
 
    # Generate the data
    X, Y = generate_time_series(my_tau=my_tau, d=my_d, n = n, mean_G1=mean_G1, mean_G2=mean_G2, mean_G3=mean_G3, c_anom = c_anom)
    X_ext, Y_ext = generate_time_series(my_tau=my_tau, d=my_d, n = n_ext,mean_G1=mean_G1, mean_G2=mean_G2, mean_G3=mean_G3,c_anom = c_anom)

    n_lab_anom = 2
    if np.sum(Y[:n_old] == 1) < n_lab_anom:
        raise ValueError(f"There have to be at least {n_lab_anom} true anomalies in the old data but there are only {np.sum(Y[:B] == 1)} of them.")

    #Maximum number of tries to get the labeled subset we want:
    num_tries_max = 100
    
    #Number of tries so far:
    num_tries_so_far = 0
    
    #Initialize the variable n_remaining_anomalies with a value different to n_lab_anom
    n_remaining_anomalies = 0
    
    #Calculate how many labels to show and how many to hide:
    n_hide = int(np.ceil((1-u)*n_old)) # if u > 0 then at least one label will be shown due to ceiling function
    n_hide_pc = (n_hide/n_old)*100
    
    #Two values used in the print output below:
    n_old_anomalies = np.sum(Y[:n_old] == 1)
    n_old_nominals = n_old - n_old_anomalies
    
    while (n_remaining_anomalies != n_lab_anom) & (num_tries_so_far < num_tries_max):
    
        #Update num_tries_so_far:
        num_tries_so_far = num_tries_so_far + 1
        
        #Randomly select n_hide of the n_old data points to mask:
        permute_indices = np.random.permutation(n_old)
        
        # The n_anomalies indices in permute_indices will correspond to the anomalies:
        hide_indices = permute_indices[0:n_hide]
        
        #Fill in Y_muted:
        Y_muted = Y[:n_old].astype(float)
        Y_muted[hide_indices] = np.nan
        n_remaining_anomalies = np.sum(Y_muted == 1)
        n_remaining_nominals = np.sum(Y_muted == 0)
        print('There were',n_old_anomalies,'anomalies and',n_old_nominals,'nominals in the initial data. After randomly masking',n_hide_pc,'% of the initial data, there remain',n_remaining_anomalies,'labeled anomalies and',n_remaining_nominals,'labeled nominals.')
    
    if (n_remaining_anomalies != n_lab_anom) & (num_tries_so_far == num_tries_max):
        raise ValueError(f"You have not obtained a random sample with {n_lab_anom} anomalies in it. Please reconsider your choice of u and n_lab_anom with respect to the value of B. Do not continue until this error message no longer occurs.")


    new_unweighted_validation_scores = np.empty((np.shape(X_ext)[0],len(models)))

    for j in range(n_loops):
        #left and right indices:
        inlef = B*j 
        inrig = B*(j+1)
        new_unweighted_validation_scores[inlef:inrig,:] = aaa.Compute_All_Model_Scores(X_ext[inlef:inrig,:],models)

    X_old = X[:n_old,:]
    Y_old = Y_muted.tolist()  
    
    X_old, X_lab, Y_lab, all_labeled_scores = aaa.init_super_sad(X_old = X_old, Y_old = Y_old, data_dim = my_d, n_data_min = 100, models=models)
    
    AAA_AUC = [0]*n_loops

    curr_L_index = n_old

    for r in range(n_loops): 

        print('We are at batch',r+1,'of',n_loops)
        
        #Get the right-hand side index too:
        curr_R_index = curr_L_index + B
        
        #Get next batch of data:
        X_new = X[curr_L_index:curr_R_index,:] 
        Y_new = Y[curr_L_index:curr_R_index].tolist()
    
        # Learn from labeled data, propose new predicted anomalies, and propose other data to label:
        X_old, X_lab, all_labeled_scores, indices_to_expert, learned_model, supervised_indices, active_indices =  aaa.super_sad(X_new = X_new, X_old = X_old, X_lab = X_lab, Y_lab = Y_lab, all_labeled_scores = all_labeled_scores, models=models, supervised_model={'class': skens.RandomForestClassifier, 'params': {'class_weight': {0: 0.999, 1: 0.001}}}, query_strategy = margin_sampling, n_data_min = 100, n_data_max = 2000, min_n_labeled = 5, n_send=n_send, pc_top = 0.4, min_n_nom=5, min_n_anom=1)
        
        # Pretend to be the expert and add the true labels to the proposed data:
        expert_provided_labels = [Y_new[j] for j in indices_to_expert]
        Y_lab = Y_lab + expert_provided_labels
        
        nY0 = Y_lab.count(0)
        nY1 = Y_lab.count(1)
        print('There are ',nY0,' labeled non-anomalies and ',nY1,' labeled anomalies so far.')

        # Store nY1 in the array
        all_nY1[curr_trial, r] = nY1
        
        #Test the current learned model on the external data in order to calculate the AUC:
        #new_preds = learned_model.predict_proba(new_unweighted_validation_scores)[:, 1]
        if learned_model != None:
            new_preds = learned_model.predict_proba(new_unweighted_validation_scores)[:,1]
            AAA_AUC[r] = roc_auc_score(Y_ext,new_preds)
            all_AAA_AUC[curr_trial,r] = roc_auc_score(Y_ext,new_preds)
        else:
            AAA_AUC[r] = 0.5
            all_AAA_AUC[curr_trial,r] = 0.5
            
        #Update indices:
        curr_L_index = curr_L_index + B

#Calculate the average AUC after each batch:
avg_AAA_AUC = np.mean(all_AAA_AUC, axis=0)

We can now plot the average AUC over time calculated on the external data set generated in each trial.

In [ ]:
xx = list(range(1,n_loops+1))

# Plot the column averages
plt.figure(figsize=(10, 6))
plt.plot(xx,avg_AAA_AUC, marker='d')

# Add plot title and labels
plt.title("Average AUC over time")
plt.xlabel("Batch", fontsize=14)
plt.ylabel("AUC", fontsize=14)

plt.xticks(ticks=list(range(5, max(xx) + 1, 5)))

# Add grid for better readability
plt.grid(True)

# Show the plot
plt.show()

And here is a plot of the cumulative number of true anomalies detected over time during each trial:

In [ ]:
# Plot each trial's nY1 values
plt.figure(figsize=(10, 6))

for trial in range(n_trials):
    plt.plot(xx, all_nY1[trial, :], marker='o', linestyle='-', label=f"Trial {trial+1}")

# Labels and title
plt.xlabel("Batch")
plt.ylabel("Number of anomalies detected")
plt.title("Cumulative number of labeled anomalies over time", fontsize=16)
plt.legend()
plt.grid(True)

# Show the plot
plt.show()